# Pysolation course project #1
27th of May 2020
Tonya White, Vera Bouwman, Louk de Mol, Elisabet Blok

So first we will import all the modules that we are going to need for this script. <br> **nibabel** is the package that we will use to import the files that we obtain from FreeSurfer. <br> **numpy** is a package that allows us to do vector and matrix operations. <br> **pandas** is a package that lets us work with large amounts of data in so-called dataframes. <br> **matplotlib.pyplot** is a package that lets us plot data. <br> **scipy.stats** is a package that lets us do statistics on our data. <br> **nibabel.freesurfer.mghformat.load** is a specific function that allows us to easily load FreeSurfer formats.

In [1]:
# Importing 'as' makes it easier for us to use a package throughout a file,
# it will allow us to use functions in that package using nib.FUNCTION() instead of using nibabel.FUNCTION()
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Importing 'from' means that we only use a specific subpart of a package
# this can be a function or a structure, like scipy.stats that contains specific functions
# instead of loading all of the functions inside scipy, 
# we can load a subpackage that only contains statistics functions
from scipy import stats
from nibabel import freesurfer
from pathlib import Path

### ORDER OF FUNCTIONALITY IN THIS SCRIPT
1. Read in IDC paths
2. Check how large output arrays should be
3. Make output data arrays
4. Array for LH thickness cases and array LH thickness controls (similar for all following outputs)
5. Make loop to cycle through all vertices 
6. Run paired ttest
7. Write output 
8. Go to next morph
9. Apply multiple testing (and figure out how!) --> check with Sander

### Loop order
1. Length of morph
2. Cycle through vertices

Why this way? To circumvent having to load the data every time

In [2]:
# Data on IDC with in the first column the ASD cases and in the second column the matched controls
# we use the following file to read the IDCs of the patients that we are going to use
idc_file_path = Path('IDC_case_control.csv')

In [3]:
# The first path refers to the folder that the subjects imaging data, thickness data, etc. are in
data_path = Path('data/')

# The second path refers to a specific sub-path that we want to use, which contains the surface data
# from FreeSurfer
surface_path = Path('surf/')

In [4]:
# This is a list of the MGH files for the thickness, area and local gyrification index of the cortex.
morph = [Path('lh.thickness'), # The left hemisphere thickness per vertex
         Path('rh.thickness'), # The right hemisphere thicknes per vertex
         Path('lh.area'), # The left hemisphere area
         Path('rh.area')] # The right hemisphere area

In [5]:
# We also select an output path that we use to store the results of our tests.
output_path = Path('output_path')
output_path.mkdir(exist_ok=True)

In [6]:
# Let's load the matched input IDCs
idc_matrix = np.genfromtxt(data_path / idc_file_path,    # The input file path
                           delimiter=',', # The delimiter that is used to split each element in the file 
                           dtype=np.str)  # The data type, in this case they are strings

In [7]:
print(idc_matrix[:, 0]) # These are the IDCs for the ASD cases

['1' '5' '8' '10']


In [8]:
print(idc_matrix[:, 1]) # These are the IDCs for the control cases

['4' '7' '12' '3']


In [9]:
# The path of morphological structures is the absolute data path, 
# the surface path and then the specific morphological path
lh_thickness_idc1_path = data_path / Path(idc_matrix[0, 0]) / surface_path / morph[0]

# We use the absolute data path that all the cases are in, each case is in a directory with their IDC,
# in this case the IDC is '1', so we get absolute_data_path/1, then we go into the surf/ directory,
# so we get absolute_data_path/1/surf/ and then we select a morphological file we want to open for this case
# which in this case is the left hemisphere thickness,
# so we get absolute_data_path/1/surf/lh.thickness.fwhm10.fsaverage.mgh
print(lh_thickness_idc1_path)

data/1/surf/lh.thickness


In [10]:
# We can now load the thickness data for this case using a function that we imported
lh_thickness_idc1_data = freesurfer.read_morph_data(lh_thickness_idc1_path)

# We can look at the shape of the data to see how many vertices there are
print(lh_thickness_idc1_data.shape)

(163842,)


In [11]:
num_idcs = idc_matrix.shape[0] # The number of IDCs is the same as the rows in the idc_matrix
num_morphs = len(morph) # The number of morphological data structures we want to analyze
num_vertices = lh_thickness_idc1_data.shape[0] # The number of vertices there are
print(f'Number of IDCs: {num_idcs}, \n'
      f'number of morphological data structures: {num_morphs},\n' 
      f'and number of vertices: {num_vertices}')

Number of IDCs: 4, 
number of morphological data structures: 4,
and number of vertices: 163842


### This is going to be the main loop, where we loop over each morphological structure

We loop over each morphological file in our morph list and then write the results of our statistical tests to MGH files in our output path

In [15]:
for morph_file in morph: # Loop over each morphological file that we have in our list
    
    print(f'The current morphological file we are analyzing is: {morph_file}')
    
    data_t = np.zeros( (num_vertices) ) # We create a three-dimensional array for t-values for each vertex
    data_p = np.zeros( (num_vertices) ) # We create a three-dimensional array for p-values for each vertex
    
    data_cases = np.zeros( (num_idcs, num_vertices) ) # We create a three-dimensional array for each case
    data_controls = np.zeros( (num_idcs, num_vertices) ) # We create a three-dimensional array for each control
    
    for idc_index in range(num_idcs): # Loop over each IDC pair, using an index for the rows in idc_matrix
        
        # Create the data path and then load the data for the case
        case_data_path = data_path / Path(idc_matrix[idc_index, 0]) / surface_path / morph_file
        case_data = freesurfer.read_morph_data(case_data_path)
        
        # Create the data path and then load the data for the control
        control_data_path = data_path / Path(idc_matrix[idc_index, 1]) / surface_path / morph_file
        control_data = freesurfer.read_morph_data(control_data_path)
        
        # Add the vertices data, based on the morphological file that we are accessing,
        # to the arrays that we just created for the cases and the controls
        data_cases[idc_index, :] = case_data
        data_controls[idc_index, :] = control_data
    
    # Look for the maximum values for the vertices, which in this case is scanning for the maximum value 
    # across the IDCs, which is why we use axis=0
    max_vert_data_cases = np.max(data_cases,
                                 axis=0)
    max_vert_data_controls = np.max(data_controls,
                                    axis=0)
    
    # Then we get the indices at which the maximum vertices for both cases and controls is above 0
    selected_vertices = np.where( (max_vert_data_cases > 0) & (max_vert_data_controls > 0))[0]
    
    # We use the indices of the vertices that are not 0 and only use those vertices to do analyses
    data_cases_selected = data_cases[:, selected_vertices]
    data_controls_selected = data_controls[:, selected_vertices]
    
    # Calculate the t-test on TWO RELATED samples of scores, a and b.
    # (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)
    # The output to this function are two arrays of size (selected_vertices, )
    # The first output are the t-values or statistics and the second output are the p_values for each vertex
    statistic, p_value = stats.ttest_rel(data_cases_selected,
                                         data_controls_selected,
                                         axis=0)
    
    # Assign the statistic, also known as the t-statistic to our data_t array
    # Assign the p values to the data_p array
    data_t[selected_vertices] = statistic
    data_p[selected_vertices] = 1 - p_value
    
    print(data_t)
    
    # Create an MGHImage using the t-statistics and the p-values
    output_ttest_t = nib.MGHImage(data_t.astype(np.float32), 
                                  affine = None)
    output_ttest_p = nib.MGHImage(data_p.astype(np.float32), 
                                  affine = None)
    
    # Save the MGHImages to the output path with an extension, based on what type of data it is.
    nib.save(output_ttest_t, 
             f'{output_path / morph_file}out_t.mgh')
    nib.save(output_ttest_p, 
             f'{output_path / morph_file}out_p.mgh')
    

The current morphological file we are analyzing is: lh.thickness
[ 1.88316912  1.19744537 -0.25838805 ...  0.46091048  1.0257171
 -0.13127626]
The current morphological file we are analyzing is: rh.thickness
[-0.4755626  -0.5134345  -0.00547918 ... -0.09016326 -0.2183428
 -0.97300827]
The current morphological file we are analyzing is: lh.area
[-1.12923819 -0.64973593  0.32310168 ... -0.05051875  2.4087538
 -0.29306811]
The current morphological file we are analyzing is: rh.area
[ 0.12789462 -0.04008177  0.46179237 ...  1.45463431  0.68636211
  0.22948028]
